<a href="https://colab.research.google.com/github/MSaydurRahman/image-processing-with-kaggle-data-set/blob/main/image_processing_with_kaggle_data_set_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path='/content/drive/My Drive/cse465/seg_train'
pred_path='/content/drive/My Drive/cse465/seg_pred/seg_pred'


In [ ]:

#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:



class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [ ]:
checkpoint=torch.load('/content/drive/My Drive/cse465/best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [ ]:
num_epochs = 2

In [ ]:
images_path=glob.glob(pred_path+'/*.jpg')

In [ ]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [ ]:
pred_dict

{'709.jpg': 'mountain',
 '7092.jpg': 'sea',
 '7036.jpg': 'street',
 '7044.jpg': 'mountain',
 '7148.jpg': 'mountain',
 '7161.jpg': 'street',
 '7013.jpg': 'sea',
 '7130.jpg': 'mountain',
 '7100.jpg': 'glacier',
 '7034.jpg': 'buildings',
 '7070.jpg': 'sea',
 '7153.jpg': 'street',
 '7069.jpg': 'sea',
 '7099.jpg': 'street',
 '7025.jpg': 'buildings',
 '7067.jpg': 'glacier',
 '7117.jpg': 'forest',
 '7124.jpg': 'glacier',
 '705.jpg': 'street',
 '7109.jpg': 'buildings',
 '7112.jpg': 'mountain',
 '7115.jpg': 'sea',
 '7133.jpg': 'sea',
 '7156.jpg': 'street',
 '7077.jpg': 'forest',
 '7104.jpg': 'buildings',
 '7050.jpg': 'buildings',
 '7043.jpg': 'mountain',
 '7028.jpg': 'street',
 '7162.jpg': 'forest',
 '7219.jpg': 'sea',
 '7190.jpg': 'sea',
 '7211.jpg': 'mountain',
 '7210.jpg': 'street',
 '720.jpg': 'forest',
 '7230.jpg': 'street',
 '7177.jpg': 'mountain',
 '7243.jpg': 'street',
 '7244.jpg': 'buildings',
 '7182.jpg': 'sea',
 '7249.jpg': 'street',
 '7186.jpg': 'sea',
 '718.jpg': 'sea',
 '7183.jpg'